# Sending JJY signal with AD9833 via FTDI
- https://en.wikipedia.org/wiki/JJY  
- https://www.analog.com/media/en/technical-documentation/data-sheets/ad9833.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', '待處理', 'ORM'],
         ['..', '..', '..', '..', '..', '..', '待處理', 'USB', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', 'Bridges', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', '待處理', 'Signal Generators', 'bitbucket', 'github', 'codes'],
         ['..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [6]:
%pylab inline 

from signal_generators import adapters, register, interfaces, shift_register, modulators
from signal_generators.ad983x.ad9833 import ad9833

from radio_clock.jjy import *

Populating the interactive namespace from numpy and matplotlib


## Debug mode?

In [7]:
adapters.SPI.DEBUG_MODE = False  # whether to show SPI written data.
ad9833.AD9833.DEBUG_MODE = False # whether to dump registers. 

## Generators and Modulators

In [10]:
with_hardware_device = False

if with_hardware_device:
    _clk = adapters.Pin.get_Ftdi_pin(pin_id = 4)
    _data = adapters.Pin.get_Ftdi_pin(pin_id = 1)

    _ss = adapters.Pin.get_Ftdi_pin(pin_id = 3)
    _spi = shift_register.ShiftRegister(stb_pin = _ss, clk_pin = _clk, data_pin = _data, polarity = 1) 
else:
    _spi = _ss = None  # using None for testing without actual hardware device.


In [11]:
ad = ad9833.AD9833(_spi, _ss)
ook = modulators.OOK(ad, freq = int(4e4))

In [12]:
ook.enable_output(True)

In [13]:
ook.enable_output(False)

In [14]:
year, month, month_day, hour, minute, second, week_day, year_day = Encoder.get_datetime_elements()
year, month, month_day, hour, minute, second, week_day, year_day    

(2020, 4, 4, 20, 18, 45, 5, 95)

## Sending JJY signal

In [ ]:
import time 

for i in range(30):
    
    t = time.time()
    print(time.localtime(t))
    
    jjy_sequence = Encoder.get_symbols_ook_sequence(t)
    ook.send_sequence(jjy_sequence)
    
#     time.sleep(5)